# K-Nearest Neighbor Lab





In [42]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff
from IPython.core.display import display

import math
import copy

## 1. (40%) Correctly implement the k-nearest neighbor (KNN) algorithm and the KNN regression algorithm

### Code requirements
- Use Euclidean distance to decide closest neighbors. 
- Include optional distance weighting for both algorithms


In [92]:
class KNNClassifier(BaseEstimator,ClassifierMixin):
    def __init__(self, columntype=[], weight_type='inverse_distance', knn=3): ## add parameters here
        """
        Args:
            columntype for each column tells you if continues[real] or if nominal[categoritcal].
            weight_type: inverse_distance voting or if non distance weighting. Options = ["no_weight","inverse_distance"]
        """
        self.columntype = columntype #Note This won't be needed until part 5
        self.weight_type = weight_type
        self.knn = knn

    def fit(self, X, y, labels=None):
        """ Fit the data; run the algorithm (for this lab really just saves the data :D)
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        self.X, self.y = X, y
        return self
    
    def predict(self, data):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """

        predictions = [None for j in range(data.shape[0])]

        for row_ind in range(data.shape[0]):
            instance = data.iloc[row_ind, :]
            distances = np.zeros(self.X.shape[0])

            for other_ind in range(self.X.shape[0]):
                other = self.X.iloc[other_ind]
                distance = self._distance(instance, other)
                distances[other_ind] = distance

            df = copy.deepcopy(self.X)
            y_copy = copy.deepcopy(self.y)
            df['distance'] = distances
            df['class'] = y_copy

            predicted_class = self._predicted_class(df)
            predictions[row_ind] = predicted_class

        return predictions

    def _distance(self, instance, other):
        num_features = instance.shape[0]
        euclidean_distance = 0


        for i in range(num_features):
            q, p = other.iloc[i], instance.iloc[i]

            if self.columntype[i] == 'nominal':
                if p != q:
                    euclidean_distance += 1
            else:
                euclidean_distance += (q - p)**2

        return math.sqrt(euclidean_distance)

    def _predicted_class(self, df):
        df = df[['distance', 'class']]
        df = df.sort_values(by=['distance'], ascending=True)
        df = df.iloc[:self.knn, :]

        if self.weight_type == 'inverse_distance':
            df['vote'] = 1 / (df['distance']**2)
        else:
            df['vote'] = 1

        df = df[['class', 'vote']]
        df = df.groupby(by='class').agg({'vote': 'sum'}) \
            .sort_values(by='vote', ascending=False) \
            .reset_index()

        return df.iloc[0,0]


    #Returns the Mean score given input data and labels
    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.
        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets
        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """
        predictions = self.predict(X)
        return (predictions == y).mean()


## 1.1 Debug and Evaluation

Debug and Evaluate your model using the parameters below:

- Use distance weighting
- KNN = 3 (three nearest neighbors)
- Don’t normalize the data
- Use Euclidean Distance

---

### 1.1.1 Debug

- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_test.arff)
- Use distance weighting
- KNN = 3 (three nearest neighbors)
- Don’t normalize the data
- Use Euclidean Distance

Expected Results:
- Not using inverse weighted distancing = roughly [68.29%]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/glass_no_inv_prediction.csv)

- Using inverse weighted distancing = roughly [74.39%]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/glass_inv_prediction.csv)


In [95]:
# Load glass data
glass_train_data = arff.loadarff('datasets/glass_train.arff')
glass_trainDF = pd.DataFrame(glass_train_data[0])
glass_trainDF.iloc[:, -1] = glass_trainDF.iloc[:, -1] \
                        .astype(str).str \
                        .split("\'", expand=True) \
                        .iloc[:,1]

display(glass_trainDF.head(2))

glass_test_data = arff.loadarff('datasets/glass_test.arff')
glass_testDF = pd.DataFrame(glass_test_data[0])
glass_testDF.iloc[:, -1] = glass_testDF.iloc[:, -1] \
                        .astype(str).str \
                        .split("\'", expand=True) \
                        .iloc[:,1]

display(glass_testDF.head(2))

# Get column types
column_type = glass_trainDF.dtypes.astype(str) \
    .map({'float64': 'continuous', 'object': 'nominal'}).tolist()
print(column_type)

# X, y
X_train = glass_trainDF.iloc[:, :-1]
y_train = glass_trainDF.iloc[:, -1]
X_test = glass_testDF.iloc[:, :-1]
y_test = glass_testDF.iloc[:, -1]

# Train on training set 
glass_knn_d_weighting = KNNClassifier(columntype=column_type)
glass_knn_nd_weighting = KNNClassifier(columntype=column_type, weight_type=None)
glass_knn_d_weighting.fit(X_train, y_train)
glass_knn_nd_weighting.fit(X_train, y_train)

# Predict on test set
glass_score_d_weighting = glass_knn_d_weighting.score(X_test, y_test)
glass_score_nd_weighting = glass_knn_nd_weighting.score(X_test, y_test)
print("Score with inverted distance weighting: %.4f" % glass_score_d_weighting)
print("Score with no inverted distance weighting: %.4f" % glass_score_nd_weighting)

,RI,Na,Mg,Al,Si,'K',Ca,Ba,Fe,Type
0,1.51793,12.79,3.50,1.12,73.03,0.64,8.77,0.0,0.0,build wind float
1,1.51643,12.16,3.52,1.35,72.89,0.57,8.53,0.0,0.0,vehic wind float


,RI,Na,Mg,Al,Si,'K',Ca,Ba,Fe,Type
0,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,headlamps
1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,build wind float


['continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'nominal']
Score with inverted distance weighting: 0.7439
Score with no inverted distance weighting: 0.6829


### 1.1.2 Evaluate

We will evaluate your model based on its performance on the [diabetes](https://archive.ics.uci.edu/ml/datasets/Diabetes) problem.
- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_test.arff)

In [13]:
# Load diabetes data

# Train on training set

# Predict on test set


## 2. (10%) Use the k-nearest neighbor algorithm (without distance weighting) for the [magic telescope](http://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope) problem

- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff) 

### 2.1
- Try it with k=3 and without normalization


In [14]:
# Load magic telescope data

# Train/Predict without normalization


### 2.2
- Try it with k=3 and with normalization (input features normalized between 0 and 1). Use the normalization formula (x-xmin)/(xmax-xmin)

In [15]:
# Train/Predict with normalization


*Discuss the accuracy results of using normalized data vs. unnormalized data*

### 2.3

- Using your normalized data, create one graph with classification accuracy on the test set over k values. 
    - Use odd values of k from 1 to 15.
- As a rough sanity check, typical knn accuracies for the magic telescope data set are 75-85%

In [16]:
# Train/Predict with normalization using k=1,3,...,15

# Graph classification accuracy over k


# For the rest of the experiments use only normalized data

## 3. (10%) Use the regression variation of your algorithm (without distance weighting) for the [housing price prediction](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) problem.

- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_test.arff).
- Use Mean Square Error (MSE) on the test set as your accuracy metric for this case.
    - Do not normalize regression output values
- Graph MSE on the test set with odd values of k from 1 to 15


In [17]:
# Load housing price prediction data

# Train/Predict using k=1,3,...,15

# Graph MSE over k


## 4. (15%) Repeat your experiments for magic telescope and housing using distance-weighted (inverse of distance squared) voting and discuss your results.


## 4.1 Magic Telescope Dataset

In [18]:
# Train/Predict magic telescope using distance-weighted voting

## 4.2 Housing Dataset

In [19]:
# Train/Predict housing using distance-weighted voting

*Discuss your results*

## 5. (10%) Use the k-nearest neighbor algorithm to solve the [credit-approval](https://archive.ics.uci.edu/ml/datasets/Credit+Approval) (credit-a) problem.

- Use this [dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/credit_approval.arff)
    - Use a 70/30 split of the data for the training/test set
- Note that this set has both continuous and nominal attributes, together with don’t know values. 
- Implement and justify a distance metric which supports continuous, nominal, and don’t know attribute values
    - You need to handle don't knows with the distance metric, not by imputing a value.
    - More information on distance metrics can be found [here](https://www.jair.org/index.php/jair/article/view/10182/24168).
- Use your own choice for k.
- As a rough sanity check, typical knn accuracies for the credit data set are 70-80%.


In [20]:
# Load dataset and split into train/test sets

# Train/Predict credit-approval


*Explain and justify your distance metric*

## 6. (15%) Use the scikit's KNN Classifier on magic telescope and KNN Regressor on housing and compare your results.

- Try out different hyperparameters to see how well you can do. 


In [21]:
# Train/Predict magic telescope using scikit's KNN

# Train/Predict housing using scikit's KNN


*Report your comparison*

## 7. (optional 5% extra credit): For the best value of k for any one of the datasets, implement a reduction algorithm that removes data points in some rational way such that performance does not drop too drastically on the test set given the reduced training set.

- Compare your performance on the test set for the reduced and non-reduced versions and give the number (and percentage) of training examples removed from the original training set. How well does your reduction algorithm work?
    - Note that performance for magic telescope is classification accuracy and for housing it is mean squared error.
    - Magic Telescope has about 12,000 instances and if you use a leave one out style of testing for your data set reduction, then your algorithm will run slow since that is n2 at each step.
    - If you wish, you may use a random subset of 2,000 of the magic telescope instances.
    - More information on reduction techniques can be found [here](http://axon.cs.byu.edu/~martinez/classes/478/slides/IBL.pdf).
